In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression,LogisticRegression,SGDRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier,XGBRegressor

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

training=pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test=pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

In [ ]:
plt.hist(training.num_sold,bins=100)
plt.show()

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))
def score(pipeline,x,y,cv):
    return -1*cross_val_score(pipeline,x,y,cv=cv,scoring=make_scorer(smape,greater_is_better=False)).mean()

In [ ]:
training[['year','month','date']]=training['date'].str.split('-',expand=True)
test[['year','month','date']]=test['date'].str.split('-',expand=True)
training=training.sample(frac=1)

In [ ]:
ohe=OneHotEncoder()
rg=LinearRegression()
xgb=XGBRegressor(n_estimators=500)
sv=SVR()
rf=RandomForestRegressor(n_estimators=500)
lgbm=LGBMRegressor()
cbr=CatBoostRegressor()
sgd=SGDRegressor()
pipeline=Pipeline(steps=[('ohe',ohe),('xgb',sv)])
print(score(pipeline,training[['year','month','country','store','product']],training.num_sold.apply(np.log1p),cv=3))
#pipeline.fit(training[['year','month','country','store','product']],training.num_sold.apply(np.log1p))
#predictions=pipeline.predict(test[['year','month','country','store','product']])
#predictions=np.exp(predictions)-1

In [ ]:
submit=pd.DataFrame({'row_id':test.row_id,'num_sold':predictions})
submit.to_csv('submission.csv',index=False)

In [ ]:
training.year=='2015'